In [1]:
import pandas as pd
import numpy as np
import sqlite3
import os

In [2]:
movies = pd.read_csv("ml-1m/movies.dat", delimiter="::", engine="python", names=["MovieID", "Title", "Genres"])
users = pd.read_csv("ml-1m/users.dat", delimiter="::", engine="python", names=["UserID", "Gender", "Age", "Occupation", "ZipCode"])
ratings = pd.read_csv("ml-1m/ratings.dat", delimiter="::", engine="python", names=["UserID", "MovieID", "Rating", "Timestamp"])

movies["Year"] = movies.Title.str[-5:-1]
movies["Year"] = movies["Year"].astype(int)
movies.Title = movies.Title.str[:-7]

In [3]:
db_name = 'movies.db'
con = sqlite3.connect(db_name)

movies.to_sql("movies", con, if_exists='replace', index=False)
users.to_sql("users", con, if_exists='replace', index=False)
ratings.to_sql("ratings", con, if_exists='replace', index=False)

In [4]:
sql_result = pd.read_sql_query("SELECT * FROM movies", con)
sql_result

,MovieID,Title,Genres,Year
0,1,Toy Story,Animation|Children's|Comedy,1995
1,2,Jumanji,Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama,1995
4,5,Father of the Bride Part II,Comedy,1995
...,...,...,...,...
3878,3948,Meet the Parents,Comedy,2000
3879,3949,Requiem for a Dream,Drama,2000
3880,3950,Tigerland,Drama,2000
3881,3951,Two Family House,Drama,2000


# 1. Wybierz wszystkie kolumny (z users)

# 2. Wybierz płeć i wiek użytkowników

# 3. Wybierz pierwszych 5 filmów.

# 4. Wybierz filmy, których ID to nie jest ani 3, ani 5.

* `<>`, `!=`, `AND`
* `IN`, `NOT`

# 5. Wybierz użytkowników, których wiek jest z przedziału [18, 35].

* `AND`, `>=`, `<=`
* `BETWEEN`

# 6. Wybierz użytkowników, których wiek jest z przedziału < 18 lub > 35.

# 7. Wybierz filmy, których rok produkcji jest parzysty, a tytuł filmu zaczyna się na literę T.

# 8. Wybierz filmy, których rok produkcji jest parzysty lub jest mniejszy od 1995.

# 9. Wybierz unikatowe grupy wiekowe użytkowników.

# 10. Wybierz unikatowe kombinacje wieku i płci użytkowników.

# 11. Wybierz zawód użytkownika, gdzie wiek zawiera się w [18, 35], ale ogranicz się do pierwszych 20 wierszy.

# 12. Wybierz filmy, mieszanka gatunków to Comedy|Romance, Action|Crime|Thriller lub Comedy|Horror

# 13. Wybierz filmy, mieszanka gatunków to Comedy|Romance, Action|Crime|Thriller lub Comedy|Horror, a rok produkcji jest większy niż 1985.

# Sortowanie

# 1. Posortuj filmy po tytule.

# 2. Posortuj filmy po tytule, ale tylko te, których tytuł zaczyna się na literę T

# 3. Wybierz unikatowe grupy wiekowe użytkowników i posortuj.

# 4. Posortuj filmy po roku produkcji malejąco, a w ramach jednego roku po tytule rosnąco.

# 5. Wybierz unikatowe kombinacje wieku i płci użytkowników i posortuj, najpierw po płci, a potem po wieku.

# 6. Wybierz unikatowe kombinacje wieku i płci użytkowników, wybierz tylko pełnoletnie kobiety i posortuj malejąco po wieku.

# Grupowanie

# 1. Wybierz liczbę filmów o danej mieszaninie gatunków.

przykładowa mieszanina: `Comedy|Romance`

# 2. Wybierz unikatowe kombinacje wieku i płci użytkowników, zlicz, ile jest rekordów dla każdej kombinacji i posortuj, najpierw po płci, a potem po wieku(a osobno po liczbie reprezentantów każdej grupy).

# 3. Wybierz minimalną, maksymalną i średnią ocenę dla każdego filmu (bez join).

# 4. Ile filmów z literą X w tytule pojawiało się w poszczególnych latach?

# 5. Wybierz lata i liczbę filmów, których tytuły zawierają literę X, ale tylko takie lata, że takich filmów było więcej niż 3.

# 6. Wybierz lata i liczbę filmów i posortuj od najpłodniejszego roku do najmniej, zostaw jedynie pierwszych 10 wierszy.

# Join

# 1. Połącz ratings z movies

# 2. Jaki jest najlepszy film wszechczasów?

* albo join i potem groupby
* albo zrobić podzapytanie które robi groupby i dopiero potem join

# 3. Bierzemy pod uwagę filmy ze >= 100 głosami

# 4. Wyświetl pełną nazwę filmu, który miał najwięcej głosów